# Looking at ETH Distribution

sources: 
- \[1\] [Ethereum blog: Launching the Ether Sale by Vitalik](https://blog.ethereum.org/2014/07/22/launching-the-ether-sale/)
- \[2\] [Ethereum AMA](https://www.reddit.com/r/IAmA/comments/2bjmgb/hi_we_are_the_ethereum_project_team_ask_us/)
- \[3\] [AVSA Tweet about contributors](https://twitter.com/avsa/status/1210587580741439489)
    

### ICO Details \[1\]:
- tranches:
    - 0-14 days: 2000 ETH per BTC
    - linear decline: 1337 ETH per BTC
- lasts 42 days
- annual issuance is 0.26x initial ETH sold
- two endowment pools (each 0.099x initial ETH sold):
    - early contributors to the project
    - long-term endowment to non-profit foundation
- reserved right to use up to 5000 BTC from pre-sale while pre-sale is running to ramp development
- exodus 3-4 multisig address: `36PrZ1KHYMpqSyAQXSG8VwbUiq2EogxLo`


## Pre-mine Analysis

### Open contributions

### Contributor Allocations

source: \[3\]
- Vitalik: Ethereum 
- Hoskinson: Classic
- Di'orio: Jaxx
- Mihai: Akasha
- Gavin: Polkadot
- Lubin: Consensys
- Jeff: Not involved anymore
- Amir: seems to be an angel investor now?
- Koblitz: cryptography professor (same as before)
- Ralph Merkle: same as above
- Stephan Tual: Slockit, Atlas Neue
- Taylor: volunteers for swarm team
- Jeremy Wood: Cardano
- Mathias: Braveno (cryptoexchange)
- Ryan: Dash
- Fierro: Mimir (lab building blockchain apps?)
- Stott: couldn’t find
- Meikle: Ethereum
- Colby: vDice
- Sureanu: Akasha
- Hidskes: couldnt find
- Mohan: Themys (blockchain risk assessment?)
- Houwu Chen: seems to be a researcher still?
- Nguyen: couldn’t find. There’s a "Forbes under 30” with his name, same guy?
- Wilding: L4, a web3 studio that built ETHGlobal
- Gluck: Riot games
- Chow: still at ethereum I believe?

### Summary